In [1]:
import boto3

region = boto3.session.Session().region_name 

account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'sagemaker-processing-container'
tag = ':latest'
processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)


In [2]:
from sagemaker import get_execution_role
role = get_execution_role()
role

'arn:aws:iam::064258348567:role/service-role/AmazonSageMaker-ExecutionRole-20200131T104492'

In [3]:
processing_repository_uri

'064258348567.dkr.ecr.us-east-1.amazonaws.com/sagemaker-processing-container:latest'

In [22]:
!docker build -t $ecr_repository .

Sending build context to Docker daemon  71.17kB
Step 1/9 : FROM python:3.7-slim-buster
 ---> e7d86653f62f
Step 2/9 : MAINTAINER ASC "mbod@asc.upenn.edu jmparelman@gmail.com"
 ---> Using cache
 ---> a2426e1e7b5b
Step 3/9 : RUN apt-get -y update && apt-get install -y --no-install-recommends         build-essential         python-dev         python3-pip     && pip3 install -U pip
 ---> Using cache
 ---> 06fc73bf297c
Step 4/9 : ENV SPACY_VERSION 3.0.5
 ---> Using cache
 ---> b382a2925541
Step 5/9 : RUN     pip3 install -U pandas          tqdm          gensim          sklearn     && pip3 install -U spacy==${SPACY_VERSION}     && python3 -m spacy download en_core_web_sm
 ---> Using cache
 ---> 2c79eee624c8
Step 6/9 : RUN mkdir -p /opt/ml/processing/input/speeches /opt/ml/processing/input/descr /opt/ml/processing/output
 ---> Using cache
 ---> 50a7cd5b3328
Step 7/9 : COPY data/omit_phrases.csv /opt/ml/input/omit_phrases.csv
 ---> Using cache
 ---> 12431191af9b
Step 8/9 : ENV PYTHONUNBUFFERED=

In [23]:
account_id

'064258348567'

In [24]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 064258348567.dkr.ecr.us-east-1.amazonaws.com


WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [25]:
!aws ecr create-repository --repository-name $ecr_repository



An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'sagemaker-processing-container' already exists in the registry with id '064258348567'


In [26]:
!docker tag {ecr_repository + tag} $processing_repository_uri
!docker push $processing_repository_uri

The push refers to repository [064258348567.dkr.ecr.us-east-1.amazonaws.com/sagemaker-processing-container]

aac9900a: Preparing 
15586ca0: Preparing 
12febf55: Preparing 
331e7578: Preparing 
3c6439bc: Preparing 
6bb9ffea: Preparing 
663e622b: Preparing 
845af46d: Preparing 
845af46d: Layer already exists latest: digest: sha256:8224d1b89f8a07ee8a0c9befdb8fe7a8e8e1dfad6a66c6d7ecd28be525a58a4e size: 2212


In [16]:
!echo $processing_repository_uri

064258348567.dkr.ecr.us-east-1.amazonaws.com/sagemaker-processing-container:latest


# can we now run the image?

In [8]:
role_arn = 'arn:aws:sts::064258348567:assumed-role/AmazonSageMaker-ExecutionRole-20200131T104492/SageMaker'

In [9]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput

script_processor = ScriptProcessor(command=['python3'],
                image_uri=processing_repository_uri,
                role=role,
                instance_count=1,
                instance_type='ml.m5.2xlarge')


In [10]:
script_processor.run(code='scripts/congress_pre_process.py',
                     inputs=[ProcessingInput(
                        source='s3://ascsagemaker/JMP_congressional_nmf/raw_data',
                        destination='/opt/ml/processing/input')],
                      outputs=[ProcessingOutput(
                        source='/opt/ml/processing/output',
                        destination='s3://ascsagemaker/JMP_congressional_nmf/dtms')],
                      arguments=['104']
                    )


Job Name:  sagemaker-processing-container-2021-05-05-14-53-53-292
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://ascsagemaker/JMP_congressional_nmf/raw_data', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-064258348567/sagemaker-processing-container-2021-05-05-14-53-53-292/input/code/congress_pre_process.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'output-1', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://ascsagemaker/JMP_congressional_nmf/dtms', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
............................./usr/local/lib/python3.7/site-packages/gen

UnexpectedStatusException: Error for Processing job sagemaker-processing-container-2021-05-05-14-53-53-292: Failed. Reason: AlgorithmError: See job logs for more information